In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726671e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### Take Random Actions in the Environment
Agent taking random actions are not able to solve the environment

In [7]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.0


### First creating models for both Actor and Critic

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [11]:
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

In [ ]:
class Actor(nn.Module):
    """Actor Model"""
    
    def __init__(self, state_size, action_size, seed, fc1_units=400, fc2_units=300, fc3_units=150):
        """Initialize parameters and build the model
        Params
            state_size (int) : Dimension of each state
            action_size (int) : Dimension of each action
            seed (int) : Ranom seed
            fc1 units (int) : number of nodes in first hidden layer
            fc2_units (int) : number of nodes in second hidden layer
            fc3_units (int) : number of nodes in third hidden layer
        """
        super(Actor, self).__init__()
        self.seed(torch.manual_seed(seed))
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, fc3_units)
        self.fc4 = nn.Linear(fc3_units, action_size)
        self.reset_parameters()
        
        def reset_parameters(self):
            self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
            self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
            self.fc3.weight.data.uniform_(*hidden_init(self.fc3))
            self.fc4.weight.data.uniform_(-3e-3, 3e-3)
        
        def forward(self, state):
            """Build an actor policy network that map states to actions"""
            x = F.relu(self.fc1(state))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            return F.tanh(self.fc4(x))

In [12]:
class Critic(nn.Module):
    """Critic Model"""
    def __init__(self, state_size, action_size, seed, f1_units=400, f2_units=300):
        """Initialize parameters and build the model
        Params
            state_size (int) : Dimension of each state
            action_size (int) : Dimension of each action
            seed (int) : Random seed
            f1_units (int) : number of nodes of first hidden layer
            f2_units (int) : number of nodes of second hidden layer
        """
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.f1 = nn.Linear(state_size, f1_units)
        self.f2 = nn.Linear(f1_units+action_size, f2_units)
        self.f3 = nn.Linear(f2_units, 1)
        self.reset_parameters()
    
    def reset_parameters(self):
        self.f1.weights.uniform_(*hidden_init(self.f1))
        self.f2.weights.uniform_(*hidden_init(self.f2))
        self.f3.weights.uniform_(-3e-3, 3e-3)
        
    def forward(self, state, action):
        """Build a critic value network that maps (state, action) pairs to Q values"""
        xs = F.relu(self.f1(state))
        x = torch.cat((xs, action), dim=1)
        x = F.relu(self.f2(x))
        return self.f3(x)